In [3]:
import pandas as pd
import numpy as np
import os 
import geopandas as gpd
import plotly.express as px
from scipy.spatial import KDTree
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 바다숲 지형정보

In [14]:
mf = gpd.read_file('./Marine_forest/Marine_forest.shp')
count = len(mf[(mf['면적'] == mf['구역']) & (mf['면적'] == mf['규모'])])
print(count)
mf.drop(columns=['구역','규모'],inplace=True)
col_names = {'시도명': 'province','시군구명':'city','시군구코드':'city_code','위도':'latitude','경도':'longitude','마을어장':'town','면적':'area'}
mf.rename(columns=col_names, inplace=True)
#기본 칼럼인 위도 경도는 오류가 있었음
mf['latitude'] = mf['geometry'].y
mf['longitude'] = mf['geometry'].x

219


바다숲 전체

In [7]:
title = '2013~2022년 바다숲 조성사업 현황'
buff = mf.loc[mf['시설년도'] <= 2022]
buff = buff.to_crs(epsg=32652)
center = {'lat': 35.8, 'lon': buff['longitude'].median()}
fig = px.scatter_mapbox(buff, lat=buff['latitude'], lon=buff['longitude'], size=buff['area'],
                        color=buff['area'], size_max=50, zoom=5, center=center, title =title)
fig.update_layout(mapbox_style="open-street-map", margin={"r":10,"t":70,"l":20,"b":20},
                  width=1000 , height=800,legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1.15))
fig.show()

특정 년도마다 해양지형정보 측정기와 바다숲 위치정보

In [12]:
title = f'2018년 해양지형정보 2013~2018바다숲'
ocean_df  = pd.read_csv('./data/2018.csv')


buff = mf.loc[mf['시설년도'] <= 2018]
buff = buff.to_crs(epsg=32652)
scaling_factor = 1
center = {'lat': 35.8, 'lon': buff['longitude'].median()}
fig = px.scatter_mapbox(buff, lat=buff['latitude'], lon=buff['longitude'], size=(buff['area']* 10000)*scaling_factor,
                        color=buff['area'], size_max=20, zoom=5, center=center, title =title)
fig.add_trace(
    go.Scattermapbox(
        lat=ocean_df['latitude'],
        lon=ocean_df['longitude'],
        mode='markers',
        marker=dict(size=5, color='red'),  # 예시로 빨간색 마커를 사용
        name='Surface Data'
    )
)
fig.update_layout(mapbox_style="open-street-map", margin={"r":10,"t":70,"l":20,"b":20},
                  width=1000 , height=800,legend=dict(orientation="h",yanchor="bottom",y=1.02,xanchor="right",x=1.15))

fig.show()


## 각 station마다의 변화

In [80]:
surface_df  = pd.read_csv('./data/surface_df.csv')

unique_values = surface_df['station'].unique().tolist()
print(unique_values)


fig = make_subplots(rows=10, cols=2)

columns_to_plot = ['총질소[μg/L]', '총인[μg/L]', '용존무기질소[μg/L]','화학적산소요구량[mg/L]','암모니아성 질소[μg/L]']
for i, d in enumerate(unique_values[:20]):
    row = i // 2 + 1  # 행 결정
    col = i % 2 + 1   # 열 결정
    x = surface_df.loc[surface_df['station'] == d].reset_index()
    
    fig.add_trace(go.Scatter(x=x['일시'], y=x['용존무기질소[μg/L]'], mode='lines+markers', name=f'Plot {d}'), row=row, col=col)

    
fig.update_layout(width=1200, height=1200, margin=dict(t=50, b=50, l=50, r=50))

fig.show()

['가로림연안1_연안', '가로림연안2_연안', '가로림연안3_연안', '가막만1_환경관리', '가막만2_환경관리', '가막만3_환경관리', '가막만4_환경관리', '가막만5_환경관리', '감포연안1_연안', '감포연안2_연안', '감포연안3_연안', '감포연안4_연안', '감포연안H1_항만', '강구연안1_연안', '강구연안2_연안', '강구연안H1_항만', '강릉연안1_연안', '강릉연안2_연안', '강릉연안3_연안', '강릉연안4_연안', '강릉연안5_연안', '강릉연안6_연안', '강릉연안7_연안', '거제도남안1_연안', '거제도남안2_연안', '거제도남안3_연안', '거제도남안4_연안', '거제도동안1_연안', '거제도동안2_연안', '거제도동안3_연안', '거제도동안4_연안', '거제도동안H1_항만', '거제도동안H2_항만', '거진연안1_연안', '거진연안2_연안', '거진연안H1_항만', '고성자란만1_연안', '고성자란만2_연안', '고성자란만3_연안', '고창연안1_연안', '고창연안2_연안', '고창연안3_연안', '고창연안4_연안', '고흥연안1_연안', '고흥연안2_연안', '고흥연안3_연안', '고흥연안4_연안', '고흥연안5_연안', '고흥연안6_연안', '고흥연안7_연안', '고흥연안8_연안', '광양만1_환경관리', '광양만2_환경관리', '광양만3_환경관리', '광양만4_환경관리', '광양만5_환경관리', '광양만6_환경관리', '광양만7_환경관리', '광양만8_환경관리', '광양만9_환경관리', '광양만H1_항만', '구룡포연안1_연안', '구룡포연안2_연안', '구룡포연안H1_항만', '군산연안1_연안', '군산연안10_연안', '군산연안2_연안', '군산연안3_연안', '군산연안4_연안', '군산연안5_연안', '군산연안6_연안', '군산연안7_연안', '군산연안8_연안', '군산연안9_연안', '기장연안1_연안', '기장연안2_연안', '기장연안3_연안', '기장연안4_연안', '기장연안H1_항만', '낙동강하구1_환경

## 2010년~ 2020년 해양환경 변화 시각화

해향지형정보 한 포인트당 겹치는 바다숲의 개수와 미치는 영향에 대해 알아봄
바다숲은 통상적으로 5년이상 되야 효과가 나타난다고 본거같음

먼저 kdTree를 활용해 좌표 거리를 계산, 특정 radius에 들어온 station을 df로 만들고 각 포인트 마다 정보의 변화를 찾기

In [13]:
from scipy.spatial import KDTree

# 특정 반경 (예: 0.01도)
radius = 0.01
surface_df = pd.read_csv('./data/surface/2018_surface_df.csv')
# surface_df의 좌표를 KDTree에 로드
tree = KDTree(surface_df[['latitude', 'longitude']].values)

# mf의 각 좌표에 대해 주어진 반경 내의 모든 surface_df의 좌표의 인덱스를 찾습니다.
indices_list = tree.query_ball_point(mf[['latitude', 'longitude']].values, r=radius)

# 모든 인덱스를 하나의 배열로 펼칩니다.
all_indices = [idx for sublist in indices_list for idx in sublist]

# 중복 제거
unique_indices = np.unique(all_indices)

# 선택된 인덱스를 사용하여 near_df 생성
near_df = surface_df.iloc[unique_indices].reset_index(drop=True)


NameError: name 'surface_df' is not defined